In [ ]:
!pip install tensorflow keras mtcnn opencv-python deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.5 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=e2e3c76754e66e3caaf18b98af684874f0997c086c1a5959b23bc71778c404b4
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire


In [ ]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from mtcnn import MTCNN
from deepface import DeepFace
from PIL import Image


25-04-23 15:54:28 - Directory /root/.deepface has been created
25-04-23 15:54:28 - Directory /root/.deepface/weights has been created


In [ ]:
facenet_model = DeepFace.build_model("Facenet")
print("FaceNet Model Loaded Successfully!")


25-04-23 15:54:40 - facenet_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facenet_weights.h5
To: /root/.deepface/weights/facenet_weights.h5
100%|██████████| 92.2M/92.2M [00:00<00:00, 113MB/s]


FaceNet Model Loaded Successfully!


In [ ]:
import zipfile
import os

zip_path = "/content/final.zip"  # Change this to your actual file name
extract_folder = "/content/dataset/"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

print("Dataset extracted successfully!")


Dataset extracted successfully!


In [ ]:
import cv2
import numpy as np
from PIL import Image
from mtcnn import MTCNN

detector = MTCNN()

def extract_face(image_path, required_size=(160, 160)):
    image = cv2.imread(image_path)
    if image is None:
        print("❌ Error reading image:", image_path)
        return None

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    faces = detector.detect_faces(image)

    if len(faces) == 0:
        return None

    # Use first face (or largest)
    face = faces[0]
    x, y, w, h = face['box']
    x, y = abs(x), abs(y)

    face_region = image[y:y+h, x:x+w]
    face_image = Image.fromarray(face_region)
    face_image = face_image.resize(required_size)
    return np.asarray(face_image)


In [ ]:
from deepface import DeepFace
from sklearn.preprocessing import normalize
import numpy as np

def get_embedding(face_pixels):
    """Extract normalized embedding using FaceNet (DeepFace)"""
    if face_pixels is None:
        print("⚠️ No face found. Skipping embedding.")
        return None
    try:
        embedding_obj = DeepFace.represent(face_pixels, model_name="Facenet", enforce_detection=False)
        embedding = np.array(embedding_obj[0]["embedding"])
        embedding = normalize(embedding.reshape(1, -1))[0]
        return embedding
    except Exception as e:
        print("❌ Embedding error:", str(e))
        return None


In [ ]:
import os
import numpy as np
import cv2
from mtcnn import MTCNN
from deepface import DeepFace
from sklearn.preprocessing import normalize
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Initialize MTCNN detector
detector = MTCNN()

# Function to extract faces from the image
def extract_face(image_path, required_size=(160, 160)):
    """Extract face using MTCNN"""
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    faces = detector.detect_faces(image)

    if len(faces) == 0:
        return None  # No face detected

    # Assuming only the first face is of interest
    x, y, width, height = faces[0]['box']
    x, y = abs(x), abs(y)

    face = image[y:y+height, x:x+width]
    face = cv2.resize(face, required_size)
    return np.array(face)

# Function to extract face embedding using DeepFace (FaceNet)
def get_embedding(face_pixels):
    """Extracts the face embedding using DeepFace (FaceNet)"""
    if face_pixels is None:
        print("⚠️ Invalid face input! Cannot extract embedding.")
        return None

    embedding = DeepFace.represent(face_pixels, model_name="Facenet", enforce_detection=False)
    return np.array(embedding[0]["embedding"])

# Load and process the dataset
dataset_path = "/content/dataset/dataset"  # Update this if needed
students = os.listdir(dataset_path)

embeddings = []
labels = []

for student in students:
    student_folder = os.path.join(dataset_path, student)

    for filename in os.listdir(student_folder):
        image_path = os.path.join(student_folder, filename)
        face = extract_face(image_path)

        if face is not None:
            embedding = get_embedding(face)
            embeddings.append(embedding)
            labels.append(student)  # Store the student's name/ID

embeddings = np.array(embeddings)
labels = np.array(labels)

# Normalize the embeddings
embeddings = normalize(embeddings)

# Train the classifier (SVM)
X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.2, random_state=42)
svm = SVC(kernel='linear', probability=True)
svm.fit(X_train, y_train)

# Test the classifier
y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy on test set: {accuracy * 100:.2f}%")

# Save the trained model (optional)
import pickle
with open("svm_model.pkl", "wb") as f:
    pickle.dump(svm, f)


Accuracy on test set: 94.44%


In [ ]:
from google.colab import files

model_path = "/content/face_recognition_model.pkl"  # Update if needed
files.download(model_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files

model_path = "/content/svm_model.pkl"  # Update if needed
files.download(model_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:

pip install opencv-python numpy scikit-learn deepface mtcnn

In [ ]:
import pickle
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder

# Assuming `embeddings` and `labels` are your feature vectors and corresponding labels
# First, train the LabelEncoder
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)  # Encode labels (e.g., students' names)

# Train the SVM classifier
model = SVC(kernel='linear', probability=True)  # You can adjust the model hyperparameters as needed
model.fit(embeddings, labels_encoded)

# Save the model and label encoder
with open('/content/svm_model.pkl', 'wb') as f:
    pickle.dump(model, f)

with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(le, f)

print("✅ Models saved successfully!")


✅ Models saved successfully!
